In [47]:
import pandas as pd
import numpy as np

# read in the data course-v1.csv
df = pd.read_csv('course-v1.csv')

In [48]:
print(df.head())

              課程名稱  課程流水號   所屬系所 授課教師  上課星期     上課節次  \
0  大學國文：文學鑑賞與寫作（一）   1001  中國文學系  汪詩珮   5.0  {7,8,9}   
1  大學國文：文學鑑賞與寫作（一）   1002  中國文學系  尹子玉   5.0  {7,8,9}   
2  大學國文：文學鑑賞與寫作（一）   1003  中國文學系  張斯翔   5.0  {7,8,9}   
3  大學國文：文學鑑賞與寫作（一）   1004  中國文學系  康韻梅   5.0  {7,8,9}   
4  大學國文：文學鑑賞與寫作（一）   1005  中國文學系  吳旻旻   5.0  {7,8,9}   

                                                課程概述  \
0  本課程於大學國文課程模組中，兼有「自我與他者」、「編劇與展演」 兩項。\r\n本課程希冀透過...   
1  本課程涵納大學國文「自我與他者」、「人類與自然」、「詮釋與記憶」等主題，以唐宋詞為閱讀範圍。...   
2  【大學國文主題設計：自我與他者、詮釋與記憶、移動與感官】\r\n\r\n1、 課程內容：本課...   
3  一、主題設計：自我與他者、個人與社會、奇幻與創造、詮釋與記憶\r\n二、課程鑑賞的文學作品主...   
4  本課程主題為「現代小說選讀」，將選讀魯迅、白先勇、奚淞、董啟章、賴香吟、駱以軍、曹麗娟、山本...   

                                                課程目標  \
0  1.通過各種代表性文本的精讀、解析及詮釋，加強學生的人文思維與涵養。 \r\n2.訓練學生分...   
1  一、通過各種代表性文本的精讀、解析及詮釋，加強學生的人文思維與涵養。\r\n二、訓練學生分析...   
2  臺大國文課的共同目標為：（本課程將視情況調整）\r\n閱讀方面──\r\n1. 通過各種代表...   
3  一、\t通過經典、代表性文本的精讀、解析及詮釋，使學生熟悉人文思維，了解人文精神與傳統在現代...   
4  1.藉由實作與討論，使學生能逐步掌握議論、說明、描寫、敘事或抒情等各種類型的寫作要點。

In [49]:
# create a new column called embedding text, which is a cancat of # "課程名稱" "課程概述" "課程目標" "課程要求"
df['embedding_text'] = df['課程名稱'] + ' ' + df['課程概述'] + ' ' + df['課程目標'] + ' ' + df['課程要求']

In [50]:
# let all Nan value in all column to be empty string
df = df.fillna('')


In [51]:
# add "passage: " to the start of each row in the embedding_text column
df['embedding_text'] = 'passage: ' + df['embedding_text']

# let the maximum length of the embedding_text column be 500
df['embedding_text'] = df['embedding_text'].apply(lambda x: str(x)[:8000] if isinstance(x, str) else x)

In [52]:
# get rid of  df['課程概述'] + ' ' + df['課程目標'] + ' ' + df['課程要求']
df = df.drop(columns=['課程概述', '課程目標', '課程要求'])

In [53]:
# let first 50 courses be a new df, df1
# df1 = df.head(50)
df1 = df


In [54]:
# load environment variables
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.vectorstores import Pinecone
from langchain.embeddings.base import Embeddings
from langchain.schema import Document


In [55]:
# !pip install -qU langchain-openai
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)
# embeddings = PineconeEmbeddings(model="multilingual-e5-large")
index_name = "ntuim-course"


In [ ]:
# upload the data to Pinecone, let the matadata be the rows of the df1, and use embedding_text to calculate the embedding

documents = []
# for _, row in df1[:100].iterrows():
for _, row in df1.iterrows():
    # Create the document with text for embeddings and rest as metadata
    metadata = row.drop('embedding_text').to_dict()
    doc = Document(page_content=row['embedding_text'], metadata=row.to_dict())
    documents.append(doc)

# Generate embeddings for the "embedding_text" using the embeddings model
# embeddings = Embeddings()  # Replace with your actual embedding model (BaichuanTextEmbeddings)

In [57]:
docsearch = PineconeVectorStore.from_documents(documents, embedding=embeddings, index_name=index_name)
# upload the embeddings to the Pinecone index